In [1]:
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.algorithms.components import strongly_connected_component_subgraphs
from networkx.algorithms.components import connected_component_subgraphs
import timeit
import os


import pickle
import time

workdir = "/home/michele/aalto/dm/DMproject/dmproject/approximate"
if(os.curdir != workdir):
    os.chdir("/home/michele/aalto/dm/DMproject/dmproject/approximate")

import sys
sys.path.insert(0, '/home/michele/aalto/dm/DMproject/dmproject/utils')

import utils

In [2]:
graph_files = []
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'wiki_vote', 'Wiki-Vote.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'epinions', 'soc-Epinions1.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'gplus', 'gplus_combined.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'soc_pokec', 'soc-pokec-relationships.txt'))
graph_names = ["wikivote", "epinions", "gplus", "pokec"]

In [ ]:

graphname = graph_names[0]
file = graph_files[0]
#fh = open(file, "r")
# import of directed
dG = nx.read_adjlist(file, create_using=nx.DiGraph())
# conversion to undirected graph
G = dG.to_undirected()

In [ ]:
def reservoir_sampling(G, largest_cc_len, n_repetitions, p_samples, directed=False):
    res = []
    for p in p_samples:
        samples_mat_p = []
        for n in n_repetitions:
            samples_count = largest_cc_len*p
            samples_mat = []
            for i in range(n):
                samples = []
                
                if directed:
                    subG = nx.DiGraph()
                else:
                    subG = nx.Graph()
                
                for t, edge in enumerate(G.edges):
                    prob = np.random.uniform()
                    threshold = min(samples_count/(t+1), 1)
                    
                    if prob <= threshold: 
                        subG.add_edge(edge[0], edge[1])
                samples_mat.append(subG)
            samples_mat_p.append(samples_mat)
        res.append(samples_mat_p)
    return res

np.random.seed(132435)
p_samples = [0.2]#, 0.3, 0.4, 0.7]
n_repetitions = [2]#, 50, 100, 500, 1000]

matrix_cc = reservoir_sampling(G, largest_cc_len, n_repetitions, p_samples)
pickle.dump(matrix_cc, open(file + "_matrix_cc_list.pkl", "wb")) 
matrix_scc = reservoir_sampling(G, largest_cc_len, n_repetitions, p_samples, directed=True)
pickle.dump(matrix_scc, open(file + "_matrix_scc_list.pkl", "wb"))

In [19]:
newG = np.array(matrix_cc)
print(newG.shape)
print(len(newG[0,0,0].edges()))

(1, 1, 2)
7382
